In [45]:
import facebook
import json
import pprint
import csv

# Facebook access token hardcoded. Get this from Facebook Graph Explorer
ACCESS_TOKEN = ""

In [46]:
# Making API call to get all posts for given ID (in this case, politician's page)
graph = facebook.GraphAPI(access_token=ACCESS_TOKEN, version='2.7')
all_posts = graph.get_object(id='leehsienloong/posts')

In [47]:
# Creating a list of post IDs from the JSON data (returned from API call)  
post_id_list= []

for post in all_posts["data"]:
    post_id_list.append(post["id"])

In [48]:
# Loop through each post, retrieve relevant post information (message, likes count, reactions count, comments count)
# Store data in list. Each element is still in JSON format.
post_desc_list = []
fields_required = "message, shares, comments.limit(0).summary(true), likes.limit(0).summary(true),reactions.type(LOVE).limit(0).summary(total_count).as(reactions_love),reactions.type(WOW).limit(0).summary(total_count).as(reactions_wow),reactions.type(HAHA).limit(0).summary(total_count).as(reactions_haha),reactions.type(SAD).limit(0).summary(total_count).as(reactions_sad),reactions.type(ANGRY).limit(0).summary(total_count).as(reactions_angry)"

for ids in post_id_list:
    post_desc_list.append(graph.get_object(id=ids, fields=fields_required))
    
print len(post_desc_list)
pprint.pprint(post_desc_list[0])

25
{u'comments': {u'data': [],
               u'summary': {u'can_comment': True,
                            u'order': u'ranked',
                            u'total_count': 11}},
 u'id': u'125845680811480_1349019458494090',
 u'likes': {u'data': [],
            u'summary': {u'can_like': True,
                         u'has_liked': False,
                         u'total_count': 625}},
 u'message': u'Here are some highlights from my #ChineseNewYear visit to Senoko Energy this morning. \u2013 LHL\n\n#CNY\n \n(PMO Video by Alex Qiu and Chiez How)',
 u'reactions_angry': {u'data': [], u'summary': {u'total_count': 0}},
 u'reactions_haha': {u'data': [], u'summary': {u'total_count': 1}},
 u'reactions_love': {u'data': [], u'summary': {u'total_count': 20}},
 u'reactions_sad': {u'data': [], u'summary': {u'total_count': 0}},
 u'reactions_wow': {u'data': [], u'summary': {u'total_count': 0}},
 u'shares': {u'count': 35}}


In [50]:
# Test Print

# print "Status:", post_desc_list[0]["message"]
# print "Comments:", post_desc_list[0]["comments"]["summary"]["total_count"]
# print "Likes:", post_desc_list[0]["likes"]["summary"]["total_count"]
# print "Reactions:"
# print "- angry", post_desc_list[0]["reactions_angry"]["summary"]["total_count"]
# print "- haha", post_desc_list[0]["reactions_haha"]["summary"]["total_count"]
# print "- love", post_desc_list[0]["reactions_love"]["summary"]["total_count"]
# print "- sad", post_desc_list[0]["reactions_sad"]["summary"]["total_count"]

status_str = str(post_desc_list[0]["message"].encode('utf8'))
comments_count = str(post_desc_list[0]["comments"]["summary"]["total_count"])
shares_count = str(post_desc_list[0]["shares"]["count"])
likes_count = str(post_desc_list[0]["likes"]["summary"]["total_count"])
angry_count = str(post_desc_list[0]["reactions_angry"]["summary"]["total_count"])
haha_count = str(post_desc_list[0]["reactions_haha"]["summary"]["total_count"])
love_count = str(post_desc_list[0]["reactions_love"]["summary"]["total_count"])
sad_count = str(post_desc_list[0]["reactions_sad"]["summary"]["total_count"])
wow_count = str(post_desc_list[0]["reactions_wow"]["summary"]["total_count"])

#print status_str

headers = ["status","comments_count","shares_count","likes_count","angry_count","haha_count","love_count","sad_count","wow_count"]
row_string = [status_str,comments_count,shares_count,likes_count,angry_count,haha_count,love_count,sad_count,wow_count]
print headers
print row_string

with open('datafile.csv', 'wb') as f:
    w = csv.writer(f, delimiter=",")
    w.writerow(headers)
    w.writerow(row_string)
    
print "done"

['status', 'comments_count', 'shares_count', 'likes_count', 'angry_count', 'haha_count', 'love_count', 'sad_count', 'wow_count']
['Here are some highlights from my #ChineseNewYear visit to Senoko Energy this morning. \xe2\x80\x93 LHL\n\n#CNY\n \n(PMO Video by Alex Qiu and Chiez How)', '11', '35', '625', '0', '1', '20', '0', '0']
done


In [55]:
data_dict = {}

for item in post_desc_list:
    data_dict["message"] = item["message"]
    data_dict["comments"] = str(item["comments"]["summary"]["total_count"])
    data_dict["likes"] = str(item["likes"]["summary"]["total_count"])
    data_dict["angry"] = str(item["reactions_angry"]["summary"]["total_count"])
    data_dict["haha"] = str(item["reactions_haha"]["summary"]["total_count"])
    data_dict["love"] = str(item["reactions_love"]["summary"]["total_count"])
    data_dict["sad"] = str(item["reactions_sad"]["summary"]["total_count"])
    data_dict["wow"] = str(item["reactions_wow"]["summary"]["total_count"])

#print data_dict

import csv

#new_dict = dict((k.encode('ascii'), v) for (k, v) in data_dict.items())
#print new_dict
with open('datafile.csv', 'wb') as f:
    w = csv.DictWriter(f, data_dict.keys())
    w.writeheader()
    w.writerow({k:v.encode('utf8') for k,v in data_dict.items()})
    
print "done"

# print "Status:", post_desc_list[0]["message"]
# print "Comments:", post_desc_list[0]["comments"]["summary"]["total_count"]
# print "Likes:", post_desc_list[0]["likes"]["summary"]["total_count"]
# print "Reactions:"
# print "- angry", post_desc_list[0]["reactions_angry"]["summary"]["total_count"]
# print "- haha", post_desc_list[0]["reactions_haha"]["summary"]["total_count"]
# print "- love", post_desc_list[0]["reactions_love"]["summary"]["total_count"]
# print "- sad", post_desc_list[0]["reactions_sad"]["summary"]["total_count"]
# print "- wow", post_desc_list[0]["reactions_wow"]["summary"]["total_count"]

AttributeError: 'list' object has no attribute 'encode'